In [ ]:
# Clona la repo e installa requisiti compatibili con NengoDL
!rm -rf /content/snn-ids && git clone https://github.com/devedale/snn-ids.git /content/snn-ids -q
%cd /content/snn-ids

# Disinstalla eventuali Keras 3 e pacchetti confliggenti
!pip -q uninstall -y keras keras-nightly tf-keras keras-preprocessing keras-vis || true

# Installa versioni testate con NengoDL + kaggle
!pip -q install "tensorflow==2.13.1" "keras==2.13.1" "nengo==3.2.0" "nengo-dl==3.6.0" kaggle pandas scikit-learn pyyaml

import tensorflow as tf, keras
print("TF:", tf.__version__, "Keras:", keras.__version__)


In [ ]:
# Fissa numpy/typing_extensions e riavvia runtime per allineare ABI
!pip -q install "numpy==1.23.5" "typing_extensions==4.8.0" "protobuf<6.0.0"
import os
print("Riavvio runtime per completare l'installazione...")
os.kill(os.getpid(), 9)


# SNN (Nengo/NengoDL) su UNSW-NB15 — Colab via Kaggle

- Scarica UNSW-NB15 con Kaggle API (`kaggle.json`)
- Preprocessing minimale
- Rete Nengo LIF + training con NengoDL



In [ ]:
# Setup Kaggle
import os, json
os.makedirs('/root/.kaggle', exist_ok=True)
# carica il tuo kaggle.json in /content/kaggle.json prima di eseguire
if os.path.exists('/content/kaggle.json'):
  !cp -f /content/kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json || true


In [ ]:
# Download UNSW-NB15 da Kaggle (aggiorna lo slug se necessario)
from pathlib import Path
Path('/content/data').mkdir(exist_ok=True)
!kaggle datasets download -d mOhammad/unsw-nb15 -p /content/data -q
!unzip -q -o /content/data/*.zip -d /content/data
!ls -lh /content/data | head -n 20

# Esempio: conversione CSV→dataset SNN (sostituisci il path esatto del CSV)
# !python -m tools.csv_to_snn_dataset --input /content/data/UNSW_NB15_training-set.csv --output /content/data/unsw_snn.csv --label-col label


In [ ]:
# Preprocessing minimo (adatta a seconda dei CSV)
import pandas as pd, numpy as np
from glob import glob
files = sorted(glob('/content/data/**/*.csv', recursive=True)) or sorted(glob('/content/data/*.csv'))
print(len(files), 'csv trovati')

dfs = []
for f in files[:5]:
  try:
    df = pd.read_csv(f)
    dfs.append(df)
  except Exception as e:
    print('skip', f, e)

df = pd.concat(dfs, ignore_index=True)
print(df.shape)

label_col = 'label' if 'label' in df.columns else ('Label' if 'Label' in df.columns else None)
assert label_col is not None, 'Colonna etichetta non trovata'
labels = df[label_col].astype(str).str.lower()
y = (~labels.str.contains('normal') & ~labels.str.contains('benign')).astype(int).values

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_candidates = [c for c in df.columns if df[c].dtype == object and c != label_col]
cat_cols = cat_candidates[:3]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_num = scaler.fit_transform(df[num_cols].fillna(0).values.astype(float)) if num_cols else np.zeros((len(df),0))
X_cat = pd.get_dummies(df[cat_cols].fillna('NA').astype(str)).values if cat_cols else np.zeros((len(df),0))
X = np.hstack([X_num, X_cat]).astype(np.float32)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
input_size, output_size = X_train.shape[1], 2
X_train.shape, X_test.shape


In [ ]:
# Nengo: rete LIF e training con NengoDL
import nengo, nengo_dl, tensorflow as tf

timesteps = 10
with nengo.Network() as net:
  inp = nengo.Node(np.zeros(input_size))
  ens1 = nengo.Ensemble(n_neurons=256, dimensions=input_size, neuron_type=nengo.LIF())
  nengo.Connection(inp, ens1)
  ens2 = nengo.Ensemble(n_neurons=output_size, dimensions=output_size, neuron_type=nengo.LIF())
  nengo.Connection(ens1, ens2)
  p_out = nengo.Probe(ens2, synapse=0.1)

X_train_enc = np.repeat(X_train[:, None, :], timesteps, axis=1)
X_test_enc  = np.repeat(X_test[:,  None, :], timesteps, axis=1)
y_train_oh = tf.keras.utils.to_categorical(y_train, num_classes=output_size)
y_test_oh  = tf.keras.utils.to_categorical(y_test,  num_classes=output_size)
y_train_enc = np.repeat(y_train_oh[:, None, :], timesteps, axis=1)
y_test_enc  = np.repeat(y_test_oh[:,  None, :], timesteps, axis=1)

minibatch = 128
with nengo_dl.Simulator(net, minibatch_size=minibatch) as sim:
  inputs = {inp: X_train_enc}
  targets = {p_out: y_train_enc}
  sim.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss=tf.keras.losses.MSE)
  sim.fit(inputs, targets, epochs=5, validation_split=0.2, verbose=2)
  test_dict = {inp: X_test_enc}
  sim.run_steps(X_test_enc.shape[1], data=test_dict)
  out = sim.data[p_out]

pred = out.mean(axis=1)
acc = (pred.argmax(axis=1) == y_test).mean()
print('Test accuracy:', acc)


In [ ]:
# Salvataggio rete Nengo
import pickle, os
os.makedirs('/content/out', exist_ok=True)
with open('/content/out/nengo_unsw_nb15.pkl', 'wb') as f:
  pickle.dump(net, f)
print('Salvato /content/out/nengo_unsw_nb15.pkl')
